# Uji Coba Code dari MediaPipe dan FaceNet

#### Import Library

In [49]:
import numpy as np
import mediapipe as mp
import dlib
import os

#### Inisiasi Wajah dan Drawing dari Media Pipe

In [57]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1, color=(255, 0, 0))  # Warna biru (BGR)


#### Convert Gambar menjadi Gambar yang ada Face Landmark MediaPipe

In [58]:
def process_image(image_path, output_folder):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(img_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Gambar landmark pada wajah (opsional)
            mp_drawing.draw_landmarks(img, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION, drawing_spec, drawing_spec)

            # Simpan landmark ke dalam file teks
            filename = os.path.basename(image_path).split('.')[0] + '.txt'
            output_path = os.path.join(output_folder, filename)
            with open(output_path, 'w') as f:
                for landmark in face_landmarks.landmark:
                    f.write(f'{landmark.x} {landmark.y} {landmark.z}\n')

            # Simpan gambar yang telah ditandai (opsional)
            output_image_path = os.path.join(output_folder, filename[:-4] + '.jpg')
            cv2.imwrite(output_image_path, img)
    else:
        print(f"Tidak ada wajah terdeteksi di {image_path}")

#### Eksport Hasil Labelling Dengan MediaPipe

In [ ]:
dataset_folder = r"\program-TA-master\Dataset\Depression"
output_folder = r"\program-TA-master\Dataset\Hasil\TestMediaPipe"

# Pastikan folder output существует
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(dataset_folder):
    if filename.endswith(('.jpg', '.png', '.jpeg')):
        image_path = os.path.join(dataset_folder, filename)
        process_image(image_path, output_folder)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '\\program-TA-master\\Dataset\\test'

In [71]:
# Path ke model dlib (ganti dengan path Anda)
predictor_path = ".\Model\shape_predictor_68_face_landmarks.dat"  # Sesuaikan dengan path Anda
detector = dlib.get_frontal_face_detector() #inisialisasi detector wajah
predictor = dlib.shape_predictor(predictor_path) #inisialisasi predictor landmark

#### DLIB Dataset

In [72]:
def process_image2(image_path, output_folder):
    try:
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error reading image: {image_path}")
            return

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        dets = detector(gray, 1)

        for det in dets:
            x1, y1, x2, y2 = det.left(), det.top(), det.right(), det.bottom()

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Gambar bounding box

            shape = predictor(gray, det)
            for i in range(68):
                x = shape.part(i).x
                y = shape.part(i).y
                cv2.circle(img, (x, y), 2, (0, 255, 255), -1)  # Gambar landmark (kuning)

            output_filename = os.path.basename(image_path)
            output_path = os.path.join(output_folder, output_filename)
            cv2.imwrite(output_path, img)

    except Exception as e:
        print(f"Error processing image {image_path}: {e}")

In [87]:
def process_image3(image_path, output_folder):
    """
    Memproses gambar untuk membuat latar belakang putih, memusatkan bounding box dan landmark,
    dan menggambar landmark dan bounding box.
    """
    try:
        # 1. Membaca gambar menggunakan OpenCV
        img = cv2.imread(image_path)

        # 2. Memeriksa apakah gambar berhasil dibaca
        if img is None:
            print(f"Error reading image: {image_path}")
            return

        # 3. Mendapatkan tinggi, lebar, dan jumlah channel gambar
        height, width, channels = img.shape

        # 4. Membuat latar belakang putih dengan ukuran yang sama seperti gambar asli
        white_bg = np.ones((height, width, channels), dtype=np.uint8) * 255

        # 5. Mengubah gambar ke grayscale untuk deteksi wajah
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # 6. Mendeteksi wajah dalam gambar grayscale
        dets = detector(gray, 1)

        # 7. Memeriksa apakah ada wajah terdeteksi
        if not dets:
            print(f"No faces detected in {image_path}")
            # 8. Jika tidak ada wajah, simpan hanya latar belakang putih
            cv2.imwrite(os.path.join(output_folder, os.path.basename(image_path)), white_bg)
            return

        # 9. Mengambil wajah pertama yang terdeteksi
        det = dets[0]

        # 10. Mendapatkan koordinat bounding box wajah
        x1, y1, x2, y2 = det.left(), det.top(), det.right(), det.bottom()

        # 11. Menghitung pusat bounding box
        center_x_box = (x1 + x2) // 2
        center_y_box = (y1 + y2) // 2

        # 12. Menghitung offset untuk memusatkan bounding box
        offset_x = width // 2 - center_x_box
        offset_y = height // 2 - center_y_box

        # 13. Menggeser koordinat bounding box ke tengah gambar
        x1 += offset_x
        y1 += offset_y
        x2 += offset_x
        y2 += offset_y

        # 14. Menggambar bounding box yang dipusatkan pada latar belakang putih
        cv2.rectangle(white_bg, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # 15. Memprediksi landmark wajah
        shape = predictor(gray, det)

        # 16. Menggambar landmark yang dipusatkan pada latar belakang putih
        for i in range(68):
            x = shape.part(i).x + offset_x
            y = shape.part(i).y + offset_y
            cv2.circle(white_bg, (x, y), 3, (0, 0, 255), -1)

        # 17. Membuat path output untuk menyimpan gambar yang diproses
        output_filename = os.path.basename(image_path)
        output_path = os.path.join(output_folder, output_filename)

        # 18. Menyimpan gambar yang telah diproses
        cv2.imwrite(output_path, white_bg)

    except Exception as e:
        # 19. Menangani kesalahan yang mungkin terjadi selama pemrosesan gambar
        print(f"Error processing image {image_path}: {e}")

In [88]:
dataset_folder = ".\Dataset\Depression"
output_folder = ".\Dataset\Hasil\TestDILB2DepressionBaru"

os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(dataset_folder):
    if filename.endswith(('.jpg', '.png', '.jpeg')):
        image_path = os.path.join(dataset_folder, filename)
        process_image3(image_path, output_folder)

print("Dataset berhasil dibuat") # Menambahkan print statement setelah loop selesai

Dataset berhasil dibuat


In [4]:
data = np.load('26_visual.npy')
print(data)

[[-1.82789519 -1.82789519 -1.77238073 ...  0.86314231  0.88679958
   0.86314231]
 [-1.82762691 -1.82762691 -1.76316704 ...  0.92324985  0.95053407
   0.89596563]
 [-1.78095242 -1.81060589 -1.78095242 ...  0.84926014  0.87542675
   0.84926014]
 ...
 [-1.75654829 -1.72948774 -1.62124552 ...  0.89631672  0.91910159
   0.89631672]
 [-1.78457107 -1.75762816 -1.67679946 ...  0.82698305  0.85020141
   0.82698305]
 [-1.79824221 -1.79824221 -1.71389146 ...  0.88683579  0.88683579
   0.86248988]]


In [5]:
print("Bentuk array:", data.shape)
print("Tipe data array:", data.dtype)

Bentuk array: (485, 136)
Tipe data array: float64


In [29]:
from PIL import Image, ImageDraw
import os
import mediapipe as mp
import cv2
from mediapipe.framework.formats import landmark_pb2  # Impor landmark_pb2

#### Rekonstruksi Gambar Dataset DAIC-WOZ 
Ekstraksi Dataset Format CLNF secara 2D menjadi 68 titik dataset

In [75]:
from PIL import Image, ImageDraw
import os

def load_clnf_features(filepath):
    """
    Membaca file CLNF_features dan mengembalikan data sebagai list.
    Setiap baris file diubah menjadi list berisi nilai-nilai float.
    """
    data = []
    with open(filepath, 'r') as file:
        first_line = True  # Flag untuk menandai baris pertama (header)
        for line in file:
            if first_line:  # Skip baris pertama
                first_line = False
                continue
            values = line.strip().split()  # Memisahkan nilai-nilai di setiap baris
            data.append([float(val.replace(',', '')) for val in values]) # Mengonversi nilai ke float dan menambahkan ke list
    return data

def extract_landmarks(frame_data):
    """
    Mengekstrak koordinat landmark (x, y) dari data frame.
    Koordinat x dan y landmark diambil dari posisi tertentu dalam data frame.
    """
    x_coords = frame_data[4:72]  # Mengambil koordinat x landmark
    y_coords = frame_data[72:]   # Mengambil koordinat y landmark
    landmarks = list(zip(x_coords, y_coords))  # Menggabungkan koordinat x dan y menjadi pasangan (x, y)
    return landmarks

def draw_landmarks_unified(image_size, landmarks, bounding_box, output_path, draw_bounding_box=False):
    """
    Menggambar landmark dan bounding box (opsional) di tengah gambar.
    Parameter draw_bounding_box menentukan apakah bounding box akan digambar atau tidak.
    """
    width, height = image_size  # Mendapatkan ukuran gambar
    image = Image.new("RGB", image_size, "white")  # Membuat gambar baru berwarna putih
    draw = ImageDraw.Draw(image)  # Membuat objek draw untuk menggambar pada gambar

    center_x = width / 2  # Menghitung pusat gambar
    center_y = height / 2

    if draw_bounding_box and bounding_box:
        # Menggambar bounding box yang dipusatkan
        x1, y1, x2, y2 = bounding_box
        box_center_x = (x1 + x2) / 2  # Menghitung pusat bounding box
        box_center_y = (y1 + y2) / 2
        offset_x = center_x - box_center_x  # Menghitung offset untuk memusatkan bounding box
        offset_y = center_y - box_center_y

        draw.rectangle((x1 + offset_x, y1 + offset_y, x2 + offset_x, y2 + offset_y), outline=(0, 255, 0), width=2) # Menggambar bounding box

        if landmarks:
            # Menggambar landmark yang dipusatkan di dalam bounding box
            for x, y in landmarks:
                draw.ellipse((x + offset_x - 3, y + offset_y - 3, x + offset_x + 3, y + offset_y + 3), fill="red") # Menggambar titik landmark
    else:
        # Menggambar landmark yang dipusatkan tanpa bounding box
        if landmarks:
            avg_x = sum(x for x, y in landmarks) / len(landmarks)  # Menghitung rata-rata koordinat x landmark
            avg_y = sum(y for x, y in landmarks) / len(landmarks)  # Menghitung rata-rata koordinat y landmark

            offset_x = center_x - avg_x  # Menghitung offset untuk memusatkan landmark
            offset_y = center_y - avg_y

            for x, y in landmarks:
                draw.ellipse((x + offset_x - 3, y + offset_y - 3, x + offset_x + 3, y + offset_y + 3), fill="red") # Menggambar titik landmark
        else:
            print("Tidak ada landmark yang diberikan.")

    image.save(output_path)  # Menyimpan gambar hasil

# Contoh penggunaan
filepath = '300_CLNF_features.txt'  # Path ke file fitur CLNF
clnf_data = load_clnf_features(filepath)  # Memuat data fitur CLNF

image_size = (1000, 1000)  # Ukuran gambar output
output_folder = 'dataset_landmarks_unified'  # Folder untuk menyimpan gambar hasil

if not os.path.exists(output_folder):
    os.makedirs(output_folder)  # Membuat folder jika belum ada

end_frame = 99  # Jumlah frame yang akan diproses
frame_counter = 1

for frame_data in clnf_data:
    frame_number = int(frame_data[0])  # Mendapatkan nomor frame dari data frame
    if frame_number >= 1 and frame_number <= end_frame:
        landmarks = extract_landmarks(frame_data)  # Mengekstrak landmark dari data frame
        output_path = os.path.join(output_folder, f'{frame_counter:02d}.png')  # Membuat path output untuk gambar

        if landmarks:
            # Menghitung bounding box dari landmark
            min_x = min(x for x, y in landmarks)
            max_x = max(x for x, y in landmarks)
            min_y = min(y for x, y in landmarks)
            max_y = max(y for x, y in landmarks)
            bounding_box = (min_x, min_y, max_x, max_y)
        else:
            bounding_box = None

        # Menggambar landmark dan bounding box (atau landmark saja) = True
        # Menggambar landmark tapi tidak mau menggambar bounding box = False
        draw_landmarks_unified(image_size, landmarks, bounding_box, output_path, draw_bounding_box=True)

        frame_counter += 1
    elif frame_number > end_frame:
        break

print(f"Dataset landmark telah dibuat di folder '{output_folder}'.") # Menampilkan pesan setelah proses selesai

Dataset landmark telah dibuat di folder 'dataset_landmarks_unified'.


In [18]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

input_folder = 'dataset_landmarks'
output_folder = 'dataset_landmarks_mediapipe'

# Membuat folder output jika belum ada
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Loop melalui setiap file di folder input
for filename in sorted(os.listdir(input_folder)):
    if filename.endswith('.png'):  # Memastikan hanya file PNG yang diproses
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        # Baca gambar menggunakan OpenCV
        image = cv2.imread(input_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Deteksi landmark wajah dengan MediaPipe
        results = face_mesh.process(image_rgb)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # Gambar landmark pada gambar
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)
                )

        # Simpan gambar yang telah ditandai dengan landmark
        cv2.imwrite(output_path, image)

print(f"Gambar dengan landmark MediaPipe telah disimpan di folder '{output_folder}'.")

Gambar dengan landmark MediaPipe telah disimpan di folder 'dataset_landmarks_mediapipe'.


In [ ]:
def load_clnf_features_3d(filepath):
    """Membaca file CLNF_features 3D dan mengembalikan data sebagai list."""
    data = []
    with open(filepath, 'r') as file:
        first_line = True
        for line in file:
            if first_line:
                first_line = False
                continue
            values = line.strip().split()
            data.append([float(val.replace(',', '')) for val in values])
    return data

def extract_landmarks_3d(frame_data):
    """Mengekstrak koordinat landmark (x, y, z) dari data frame 3D."""
    x_coords = frame_data[4:72]
    y_coords = frame_data[72:140]
    z_coords = frame_data[140:]
    landmarks = list(zip(x_coords, y_coords, z_coords))
    return landmarks

def draw_landmarks(image_size, landmarks, output_path):
    """Menggambar titik-titik landmark pada gambar."""
    image = Image.new("RGB", image_size, "white")
    draw = ImageDraw.Draw(image)

    # Hitung pusat landmark
    x_coords, y_coords, _ = zip(*landmarks)
    center_x = sum(x_coords) / len(x_coords)
    center_y = sum(y_coords) / len(y_coords)

    # Hitung offset
    offset_x = image_size[0] / 2 - center_x
    offset_y = image_size[1] / 2 - center_y

    for x, y, _ in landmarks:
        # Sesuaikan koordinat dengan offset
        adjusted_x = x + offset_x
        adjusted_y = y + offset_y
        draw.ellipse((adjusted_x - 5, adjusted_y - 5, adjusted_x + 5, adjusted_y + 5), fill="blue")

    image.save(output_path)

def draw_landmarks2(image_size, landmarks, bounding_box, output_path):
    """Menggambar landmark wajah dan bounding box di tengah gambar."""
    width, height = image_size
    image = Image.new("RGB", image_size, "white")
    draw = ImageDraw.Draw(image)

    center_x = width / 2
    center_y = height / 2

    # Pusat Bounding Box
    if bounding_box:
        x1, y1, x2, y2 = bounding_box
        box_center_x = (x1 + x2) / 2
        box_center_y = (y1 + y2) / 2
        offset_x = center_x - box_center_x
        offset_y = center_y - box_center_y

        # Gambar Bounding Box yang dipusatkan
        draw.rectangle((x1 + offset_x, y1 + offset_y, x2 + offset_x, y2 + offset_y), outline="green", width=2)

        # Gambar Landmark yang dipusatkan
        if landmarks:
            for x, y in landmarks:
                draw.ellipse((x + offset_x - 3, y + offset_y - 3, x + offset_x + 3, y + offset_y + 3), fill="red")
    else:
        print("Tidak ada bounding box yang diberikan.")
        if landmarks:
            avg_x = sum(x for x, y in landmarks) / len(landmarks)
            avg_y = sum(y for x, y in landmarks) / len(landmarks)

            offset_x = center_x - avg_x
            offset_y = center_y - avg_y

            for x,y in landmarks:
                draw.ellipse((x + offset_x - 3, y + offset_y -3, x+offset_x + 3 , y + offset_y + 3), fill = "red")
    image.save(output_path)

# Contoh penggunaan
filepath = '300_CLNF_features3D.txt'  # Ganti dengan path file 3D Anda
clnf_data = load_clnf_features_3d(filepath)

image_size = (1000, 1000)
output_folder = 'dataset_landmarks_3d_centered'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

end_frame = 99  # Ubah sesuai kebutuhan
frame_counter = 1

for frame_data in clnf_data:
    frame_number = int(frame_data[0])
    if frame_number >= 1 and frame_number <= end_frame:
        landmarks = extract_landmarks_3d(frame_data)
        output_path = os.path.join(output_folder, f'{frame_counter:02d}.png')
        draw_landmarks(image_size, landmarks, output_path)
        frame_counter += 1
    elif frame_number > end_frame:
        break

print(f"Dataset landmark 3D telah dipusatkan dan dibuat di folder '{output_folder}'.")

Dataset landmark 3D telah dipusatkan dan dibuat di folder 'dataset_landmarks_3d_centered'.


In [30]:
import mediapipe as mp
import cv2
import os
from PIL import Image, ImageDraw

# Fungsi-fungsi load_clnf_features_3d dan extract_landmarks_3d sama seperti sebelumnya
# ...

def load_clnf_features_3d(filepath):
    """Membaca file CLNF_features 3D dan mengembalikan data sebagai list."""
    data = []
    with open(filepath, 'r') as file:
        first_line = True
        for line in file:
            if first_line:
                first_line = False
                continue
            values = line.strip().split()
            data.append([float(val.replace(',', '')) for val in values])
    return data

def extract_landmarks_3d(frame_data):
    """Mengekstrak koordinat landmark (x, y, z) dari data frame 3D."""
    x_coords = frame_data[4:72]
    y_coords = frame_data[72:140]
    z_coords = frame_data[140:]
    landmarks = list(zip(x_coords, y_coords, z_coords))
    return landmarks

def map_to_mediapipe_landmarks(landmarks_3d):
    """Memetakan 68 landmark 3D ke beberapa landmark MediaPipe (perkiraan)."""
    mediapipe_landmarks = []
    for x, y, z in landmarks_3d:
        landmark = landmark_pb2.NormalizedLandmark()
        landmark.x = x
        landmark.y = y
        landmark.z = z
        mediapipe_landmarks.append(landmark)
    return mediapipe_landmarks

def visualize_mediapipe_face_mesh(image_size, landmarks_3d, output_path):
    """Memvisualisasikan face mesh MediaPipe (perkiraan)."""
    image = Image.new("RGB", image_size, "white")
    draw = ImageDraw.Draw(image)

    mediapipe_landmarks = map_to_mediapipe_landmarks(landmarks_3d)

    for landmark in mediapipe_landmarks:
        x, y, _ = int(landmark.x * image_size[0]), int(landmark.y * image_size[1]), landmark.z
        draw.ellipse((x - 5, y - 5, x + 5, y + 5), fill="green")

    image.save(output_path)


# Contoh penggunaan
filepath = '300_CLNF_features3D.txt'
clnf_data = load_clnf_features_3d(filepath)

image_size = (1000, 1000)
output_folder = 'dataset_mediapipe_mesh'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

end_frame = 99
frame_counter = 1

for frame_data in clnf_data:
    frame_number = int(frame_data[0])
    if frame_number >= 1 and frame_number <= end_frame:
        landmarks_3d = extract_landmarks_3d(frame_data)
        output_path = os.path.join(output_folder, f'{frame_counter:02d}.png')
        visualize_mediapipe_face_mesh(image_size, landmarks_3d, output_path)
        frame_counter += 1
    elif frame_number > end_frame:
        break

print(f"Dataset MediaPipe Face Mesh (perkiraan) telah dibuat di folder '{output_folder}'.")

Dataset MediaPipe Face Mesh (perkiraan) telah dibuat di folder 'dataset_mediapipe_mesh'.


In [34]:
pip list

Package               Version
--------------------- -----------
absl-py               2.1.0
asttokens             3.0.0
attrs                 25.1.0
cffi                  1.17.1
cmake                 3.31.6
colorama              0.4.6
comm                  0.2.2
contourpy             1.3.1
cycler                0.12.1
debugpy               1.8.12
decorator             5.2.1
exceptiongroup        1.2.2
executing             2.2.0
flatbuffers           25.2.10
fonttools             4.56.0
ipykernel             6.29.5
ipython               8.33.0
jax                   0.5.1
jaxlib                0.5.1
jedi                  0.19.2
jupyter_client        8.6.3
jupyter_core          5.7.2
kaleido               0.2.1
kiwisolver            1.4.8
matplotlib            3.10.1
matplotlib-inline     0.1.7
mediapipe             0.10.21
ml_dtypes             0.5.1
narwhals              1.29.0
nest-asyncio          1.6.0
numpy                 1.26.4
opencv-contrib-python 4.11.0.86
opt_einsum          

You should consider upgrading via the 'c:\Users\Muhammad Daffa ZW\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
